## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-01-01-14-00 +0000,nypost,Exiled Iranian royals celebrate death of supre...,https://nypost.com/2026/02/28/us-news/exiled-i...
1,2026-03-01-01-13-45 +0000,nyt,Live Updates: Trump Says Iran’s Supreme Leader...,https://www.nytimes.com/live/2026/02/28/world/...
2,2026-03-01-01-12-03 +0000,wapo,Democrats demand immediate vote to restrain Tr...,https://www.washingtonpost.com/politics/2026/0...
3,2026-03-01-01-07-03 +0000,startribune,Iran's supreme leader has been killed during m...,https://www.startribune.com/witnesses-say-they...
4,2026-03-01-01-05-39 +0000,nyt,Elite Doctors Served Jeffrey Epstein While Tre...,https://www.nytimes.com/2026/02/28/us/jeffrey-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2591/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,iran,84
13,trump,47
38,strikes,31
24,attack,21
25,israel,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...,230
15,2026-03-01-00-39-51 +0000,bbc,What we know so far: Supreme Leader Khamenei k...,https://www.bbc.com/news/articles/cx2dyz6p3weo...,216
3,2026-03-01-01-07-03 +0000,startribune,Iran's supreme leader has been killed during m...,https://www.startribune.com/witnesses-say-they...,209
6,2026-03-01-01-04-15 +0000,wapo,Live updates: Iran’s supreme leader killed dur...,https://www.washingtonpost.com/world/2026/02/2...,204
178,2026-02-28-10-14-09 +0000,nypost,"Read, watch President Trump’s full speech afte...",https://nypost.com/2026/02/28/us-news/read-pre...,203


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,230,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...
3,125,2026-03-01-01-07-03 +0000,startribune,Iran's supreme leader has been killed during m...,https://www.startribune.com/witnesses-say-they...
201,75,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
42,48,2026-02-28-23-15-29 +0000,nyt,See a Breakdown of Ayatollah Khamenei and Othe...,https://www.nytimes.com/2026/02/28/world/middl...
138,46,2026-02-28-15-04-01 +0000,nypost,Saudi Arabia condemns Iran’s attacks on Gulf n...,https://nypost.com/2026/02/28/us-news/saudi-ar...
194,32,2026-02-28-07-10-48 +0000,nypost,"US, Israel launch ‘Operation Epic Fury’ attack...",https://nypost.com/2026/02/28/us-news/us-israe...
104,31,2026-02-28-18-40-55 +0000,nypost,Nancy Mace bashes Ilhan Omar over Iran attack ...,https://nypost.com/2026/02/28/us-news/nancy-ma...
215,29,2026-02-28-02-56-16 +0000,nyt,"Near the White House, Hundreds of High School ...",https://www.nytimes.com/2026/02/27/us/politics...
8,28,2026-03-01-01-00-00 +0000,wsj,Inheritance is one of the last reliable ways f...,https://www.wsj.com/economy/housing/in-califor...
126,27,2026-02-28-17-00-00 +0000,wsj,What Images of Kim Jong Un’s Daughter at Milit...,https://www.wsj.com/world/asia/kim-jong-un-dau...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
